In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))

from multiversx_sdk import Address, ApiNetworkProvider

from utils.utils_tx import endpoint_call
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

from context import Context

context = Context()

# Needed for ShadowFork
api_network_provider = ApiNetworkProvider('https://api.multiversx.com') if "shadowfork" in config.DEFAULT_PROXY else context.network_provider.api

print(f'Deployer account: {context.deployer_account.address.to_bech32()}')
contracts = api_network_provider.do_get_generic(f'accounts/{context.deployer_account.address.to_bech32()}/contracts?size={500}')

print(f'Total number of contracts: {len(contracts)}')

for contract in contracts:
        contract_address = contract['address']
        contract_details = context.network_provider.api.do_get_generic(f'accounts/{contract_address}?fields=developerReward')
        developer_reward = int(contract_details['developerReward'])
        print(f'Contract: {contract_address} -> developer reward: {developer_reward}')
        if developer_reward > 0:
            tx_hash = endpoint_call(context.network_provider.proxy, 20000000, context.deployer_account, Address.new_from_bech32(contract_address), "ClaimDeveloperRewards", [])